### Install Prerequisite Libraries

In [ ]:
!pip install transformers
!pip install datasets

### Load Datasets with HuggingFace's datasets Library

In [1]:
import pandas as pd
import numpy as np

In [3]:
from datasets import load_dataset

kmhas = load_dataset("jeanlee/kmhas_korean_hate_speech")
unsmile = load_dataset('smilegate-ai/kor_unsmile')

Found cached dataset kmhas_korean_hate_speech (/Users/hyunjun/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/17406fbed45548c92e0795df0675e21fb2a09ceaa098bd5ff58c7fdc7f8a63d4)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset parquet (/Users/hyunjun/.cache/huggingface/datasets/smilegate-ai___parquet/smilegate-ai--kor_unsmile-e0f75c6e3be1af78/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

### Preprocess the datasets

In [4]:
combined = []

for data in kmhas:
  for row in zip(kmhas[data]['text'], kmhas[data]['label']):
    if len(row[1]) == 1 and 8 in row[1]:
      combined.append((row[0], 0))
    else:
      combined.append((row[0], 1))

for data in unsmile:
  for row in zip(unsmile[data]['문장'], unsmile[data]['clean']):
    if row[1] == 1:
      combined.append((row[0], 0))
    else:
      combined.append((row[0], 1))

dataset = pd.DataFrame(data=combined, columns=['document', 'label'])
dataset['label'].value_counts()
#dataset.to_csv('dataset.csv')

0    64289
1    64145
Name: label, dtype: int64

In [4]:
def train_validate_test_split(df, train_percent=.7, validate_percent=.2):
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [5]:
train_data, validate_data, test_data = train_validate_test_split(dataset)

In [6]:
print(len(train_data), len(validate_data), len(test_data))

89903 25686 12845


In [7]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [8]:
print(len(train_data))

89903


### Tokenize the data into wordpiece with BertTokenizer

In [2]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")

In [10]:
val_data = validate_data.dropna(how = 'any')

In [11]:
print(len(val_data))

25686


In [ ]:
X_train_list = train_data['document'].tolist()
X_val_list = validate_data['document'].tolist()
y_train = train_data['label'].tolist()
y_val = validate_data['label'].tolist()

In [ ]:
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_val = tokenizer(X_val_list, truncation=True, padding=True)

### Fine-tune the KLUE Bert with keras

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_val),
    y_val
))

In [3]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.hf_compute_loss

<bound method TFSequenceClassificationLoss.hf_compute_loss of <transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification object at 0x7f5eda1e6aa0>>

In [ ]:
tf.device(0)

callback_earlystop = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2
)

model.fit(
    train_dataset.shuffle(10000).batch(32), epochs=5, batch_size=64,
    validation_data = val_dataset.shuffle(10000).batch(64),
    callbacks = [callback_earlystop]
)

Epoch 1/5
2810/2810 [==============================] - 615s 202ms/step - loss: 0.3489 - accuracy: 0.8433 - val_loss: 0.3171 - val_accuracy: 0.8654
Epoch 2/5
2810/2810 [==============================] - 532s 189ms/step - loss: 0.2379 - accuracy: 0.8997 - val_loss: 0.3196 - val_accuracy: 0.8663
Epoch 3/5
2810/2810 [==============================] - 531s 189ms/step - loss: 0.1435 - accuracy: 0.9438 - val_loss: 0.4559 - val_accuracy: 0.8609


In [ ]:
model.evaluate(val_dataset.batch(1024))

26/26 [==============================] - 33s 1s/step - loss: 0.4559 - accuracy: 0.8609


[0.45586568117141724, 0.8609359264373779]

### Save the fine-tuned model

In [ ]:
model.save_pretrained('curse_detection/bert-base')
model.config.to_json_file("config.json")
tokenizer.save_pretrained('curse_detection/bert-base')

('curse_detection/bert-base/tokenizer_config.json',
 'curse_detection/bert-base/special_tokens_map.json',
 'curse_detection/bert-base/vocab.txt',
 'curse_detection/bert-base/added_tokens.json',
 'curse_detection/bert-base/tokenizer.json')

### Load and Test

In [4]:
from transformers import TextClassificationPipeline

# 로드하기
loaded_tokenizer = BertTokenizerFast.from_pretrained('curse_detection/klue-bert-base')
loaded_model = TFBertForSequenceClassification.from_pretrained('curse_detection/klue-bert-base', output_attentions=True)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True,
    device=0
)

Some layers from the model checkpoint at curse_detection/klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at curse_detection/klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
/Users/hyunjun/miniconda/envs/nlp/lib/python3.8/

In [5]:
def classify_with_better_output(text_classifier, test_document):
  output = text_classifier(test_document)[0]
  clean = output[0]['score']
  curse = output[1]['score']
  print(f'{test_document} 가 입력되었으며,')
  if clean > curse:
    print(f'모델은 이 문장을 {clean * 100}% 확률로 깨끗한 문장이라고 추론했습니다.')
  else:
    print(f'모델은 이 문장을 {curse * 100}% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.')

In [11]:
classify_with_better_output(text_classifier, '시바견은 너무 귀엽다.')

시바견은 너무 귀엽다. 가 입력되었으며,
모델은 이 문장을 99.96166229248047% 확률로 깨끗한 문장이라고 추론했습니다.


In [12]:
classify_with_better_output(text_classifier, '시발자동차는 1955년에 출시된 우리나라 최초의 자동차이다.')

시발자동차는 1955년에 출시된 우리나라 최초의 자동차이다. 가 입력되었으며,
모델은 이 문장을 93.24093461036682% 확률로 깨끗한 문장이라고 추론했습니다.


In [15]:
classify_with_better_output(text_classifier, '수박씨 발아는 심은 후 10~15일 후 진행된다.')

수박씨 발아는 심은 후 10~15일 후 진행된다. 가 입력되었으며,
모델은 이 문장을 99.95315074920654% 확률로 깨끗한 문장이라고 추론했습니다.


In [16]:
classify_with_better_output(text_classifier, '만두 몇 개 시키고 옴')

만두 몇 개 시키고 옴 가 입력되었으며,
모델은 이 문장을 99.92907047271729% 확률로 깨끗한 문장이라고 추론했습니다.


In [15]:
classify_with_better_output(text_classifier, '내 만두 가져가지 마 개시키야')

내 만두 가져가지 마 개시키야 가 입력되었으며,
모델은 이 문장을 99.18236136436462% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [9]:
classify_with_better_output(text_classifier, '배고픈데 앞에서 만두 먹네 개시키')

배고픈데 앞에서 만두 먹네 개시키 가 입력되었으며,
모델은 이 문장을 99.68923926353455% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [19]:
classify_with_better_output(text_classifier, '아니 ㅅㅂ 네이버 메인 왜 이렇게 바꿨냐고')

아니 ㅅㅂ 네이버 메인 왜 이렇게 바꿨냐고 가 입력되었으며,
모델은 이 문장을 89.85130786895752% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [20]:
classify_with_better_output(text_classifier, '조밥나물 조팝나무는 실제로 있다.')

조밥나물 조팝나무는 실제로 있다. 가 입력되었으며,
모델은 이 문장을 96.74884080886841% 확률로 깨끗한 문장이라고 추론했습니다.


In [11]:
classify_with_better_output(text_classifier, '너가 시발점이야.')

너가 시발점이야. 가 입력되었으며,
모델은 이 문장을 99.9687671661377% 확률로 깨끗한 문장이라고 추론했습니다.


In [19]:
classify_with_better_output(text_classifier, '2023-1학기 텍스트마이닝 기말 텀프로젝트')

2023-1학기 텍스트마이닝 기말 텀프로젝트 가 입력되었으며,
모델은 이 문장을 99.48399662971497% 확률로 깨끗한 문장이라고 추론했습니다.


In [6]:
classify_with_better_output(text_classifier, '호의가 계속되면 권리인 줄 안다.')

호의가 계속되면 권리인 줄 안다. 가 입력되었으며,
모델은 이 문장을 99.00529980659485% 확률로 깨끗한 문장이라고 추론했습니다.


In [7]:
classify_with_better_output(text_classifier, '새끼 고양이는 귀엽다.')

새끼 고양이는 귀엽다. 가 입력되었으며,
모델은 이 문장을 99.96494054794312% 확률로 깨끗한 문장이라고 추론했습니다.


In [8]:
classify_with_better_output(text_classifier, '오리 한 마리를 구조했더니 새끼가 9마리 생겼다.')

오리 한 마리를 구조했더니 새끼가 9마리 생겼다. 가 입력되었으며,
모델은 이 문장을 99.94308352470398% 확률로 깨끗한 문장이라고 추론했습니다.


In [15]:
classify_with_better_output(text_classifier, '^^ㅣ발 우리 미드 뭐하냐고')

^^ㅣ발 우리 미드 뭐하냐고 가 입력되었으며,
모델은 이 문장을 98.60266447067261% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [14]:
classify_with_better_output(text_classifier, '좋은 하루입니다.^^ 김밥 한 줄 놓고 갑니다 @)))))))')

좋은 하루입니다.^^ 김밥 한 줄 놓고 갑니다 @))))))) 가 입력되었으며,
모델은 이 문장을 99.94366765022278% 확률로 깨끗한 문장이라고 추론했습니다.


In [21]:
classify_with_better_output(text_classifier, 'ㅅ!발놈들아 내가 그렇게 만만해보여?')

ㅅ!발놈들아 내가 그렇게 만만해보여? 가 입력되었으며,
모델은 이 문장을 99.61768388748169% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [22]:
labels_test = []
alpha_error = []
beta_error = []
test = zip(test_data['document'] , test_data['label'])
for i, (doc, label) in enumerate(test):

  if i == 1000:
    break

  output = text_classifier(doc)[0]
  clean = output[0]['score']
  curse = output[1]['score']
  result = 1 if curse > clean else 0
  if (result == label):
    labels_test.append(1)
  else:
    labels_test.append(0)
  if (result != label and label == 1):
    alpha_error.append(1)
  if (result != label and label == 0):
    beta_error.append(1)
  

print(f'Accuracy Percentage with unseen data : {(sum(labels_test) / len(labels_test)) * 100}%')
print(f'Alpha Error Percentage with unseen data : {(sum(alpha_error) / sum(labels_test)) * 100}%')
print(f'Beta Error Percentage with unseen data : {(sum(beta_error) / sum(labels_test)) * 100}%')

Accuracy Percentage with unseen data : 94.89999999999999%
Alpha Error Percentage with unseen data : 3.2665964172813484%
Beta Error Percentage with unseen data : 2.107481559536354%


In [ ]:
!tar -cvf 3rd_klue_bert_finetuned curse_detection/

# validation, application.

curse_detection/
curse_detection/bert-base/
curse_detection/bert-base/special_tokens_map.json
curse_detection/bert-base/tf_model.h5
curse_detection/bert-base/config.json
curse_detection/bert-base/tokenizer_config.json
curse_detection/bert-base/vocab.txt
curse_detection/bert-base/tokenizer.json


# Hugging Face's transformers 라이브러리로 모델 불러와 사용하기

In [1]:
from transformers import TextClassificationPipeline
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizerFast

# 파인튜닝 된 모델 로드하기
loaded_tokenizer = BertTokenizerFast.from_pretrained('Tolerblanc/klue-bert-finetuned')
loaded_model = TFBertForSequenceClassification.from_pretrained('Tolerblanc/klue-bert-finetuned', output_attentions=True)

# 기존 모델 로드하기
# loaded_tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')
# loaded_model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', output_attentions=True)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True,
    device=0
)

# 추론 테스트는 위 classify_with_better_output 적용!

/Users/hyunjun/miniconda/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some layers from the model checkpoint at Tolerblanc/klue-bert-finetuned were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model c